In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox, scrolledtext
import threading
import time
import random


class MultithreadedSortingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Multithreaded Sorting Application")
        self.root.geometry("900x750")
        self.root.configure(bg="#f0f0f0")
        
        # Data storage
        self.original_list = []
        self.sublist1 = []
        self.sublist2 = []
        self.sorted_sublist1 = []
        self.sorted_sublist2 = []
        self.final_sorted_list = []
        
        # Shared data and locks
        self.sorted_sublists = {}
        self.lock = threading.Lock()
        
        self.setup_ui()
    
    def setup_ui(self):
        title_frame = tk.Frame(self.root, bg="#2c3e50", pady=15)
        title_frame.pack(fill=tk.X)
        
        tk.Label(
            title_frame,
            text="Multithreaded Sorting Application",
            font=("Arial", 18, "bold"),
            bg="#2c3e50",
            fg="white"
        ).pack()
        
        # Main container
        main_frame = tk.Frame(self.root, bg="#f0f0f0", padx=20, pady=10)
        main_frame.pack(fill=tk.BOTH, expand=True)
        
        # Input Section
        input_frame = tk.LabelFrame(
            main_frame,
            text="Input",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        input_frame.pack(fill=tk.X, pady=(0, 10))
        
        tk.Label(
            input_frame,
            text="Enter integers (comma-separated):",
            font=("Arial", 10),
            bg="#f0f0f0"
        ).grid(row=0, column=0, sticky=tk.W, pady=5)
        
        self.entry = tk.Entry(input_frame, font=("Arial", 11), width=60)
        self.entry.grid(row=0, column=1, padx=10, pady=5, sticky=tk.EW)
        self.entry.insert(0, "7, 12, 19, 3, 18, 4, 2, 6, 15, 8")
        
        # Buttons
        button_frame = tk.Frame(input_frame, bg="#f0f0f0")
        button_frame.grid(row=1, column=0, columnspan=2, pady=10)
        
        tk.Button(
            button_frame,
            text="Start Sorting",
            command=self.start_sorting,
            bg="#27ae60",
            fg="white",
            font=("Arial", 11, "bold"),
            padx=20,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame,
            text="Generate Random",
            command=self.generate_random,
            bg="#3498db",
            fg="white",
            font=("Arial", 11),
            padx=15,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=5)
        
        tk.Button(
            button_frame,
            text="Clear",
            command=self.clear_all,
            bg="#e74c3c",
            fg="white",
            font=("Arial", 11),
            padx=20,
            pady=5,
            cursor="hand2"
        ).pack(side=tk.LEFT, padx=5)
        
        input_frame.columnconfigure(1, weight=1)
        
        # Visualization Section
        viz_frame = tk.LabelFrame(
            main_frame,
            text="Visualization",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        viz_frame.pack(fill=tk.BOTH, expand=True, pady=(0, 10))
        
        # Create canvas for visualization
        canvas_frame = tk.Frame(viz_frame, bg="white", relief=tk.SUNKEN, bd=2)
        canvas_frame.pack(fill=tk.BOTH, expand=True, pady=5)
        
        self.canvas = tk.Canvas(canvas_frame, bg="white", height=300)
        self.canvas.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Arrays Display
        display_frame = tk.Frame(viz_frame, bg="#f0f0f0")
        display_frame.pack(fill=tk.X, pady=5)
        
        # Original List
        tk.Label(
            display_frame,
            text="Original List:",
            font=("Arial", 10, "bold"),
            bg="#f0f0f0",
            anchor=tk.W
        ).grid(row=0, column=0, sticky=tk.W, pady=3)
        
        self.original_label = tk.Label(
            display_frame,
            text="",
            font=("Arial", 10),
            bg="#ecf0f1",
            relief=tk.SUNKEN,
            anchor=tk.W,
            padx=5,
            pady=5
        )
        self.original_label.grid(row=0, column=1, sticky=tk.EW, padx=5, pady=3)
        
        # Sublist 1 (Sorting Thread 0)
        tk.Label(
            display_frame,
            text="Sorting Thread 0:",
            font=("Arial", 10, "bold"),
            bg="#f0f0f0",
            anchor=tk.W
        ).grid(row=1, column=0, sticky=tk.W, pady=3)
        
        self.sublist1_label = tk.Label(
            display_frame,
            text="",
            font=("Arial", 10),
            bg="#e8f8f5",
            relief=tk.SUNKEN,
            anchor=tk.W,
            padx=5,
            pady=5
        )
        self.sublist1_label.grid(row=1, column=1, sticky=tk.EW, padx=5, pady=3)
        
 
        tk.Label(
            display_frame,
            text="Sorting Thread 1:",
            font=("Arial", 10, "bold"),
            bg="#f0f0f0",
            anchor=tk.W
        ).grid(row=2, column=0, sticky=tk.W, pady=3)
        
        self.sublist2_label = tk.Label(
            display_frame,
            text="",
            font=("Arial", 10),
            bg="#fef9e7",
            relief=tk.SUNKEN,
            anchor=tk.W,
            padx=5,
            pady=5
        )
        self.sublist2_label.grid(row=2, column=1, sticky=tk.EW, padx=5, pady=3)
        
        # Final Sorted List
        tk.Label(
            display_frame,
            text="Sorted List:",
            font=("Arial", 10, "bold"),
            bg="#f0f0f0",
            anchor=tk.W
        ).grid(row=3, column=0, sticky=tk.W, pady=3)
        
        self.sorted_label = tk.Label(
            display_frame,
            text="",
            font=("Arial", 10),
            bg="#d5f4e6",
            relief=tk.SUNKEN,
            anchor=tk.W,
            padx=5,
            pady=5
        )
        self.sorted_label.grid(row=3, column=1, sticky=tk.EW, padx=5, pady=3)
        
        display_frame.columnconfigure(1, weight=1)
        
        # Thread Status Section
        thread_frame = tk.LabelFrame(
            viz_frame,
            text="Active Threads",
            font=("Arial", 11, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        thread_frame.pack(fill=tk.X, pady=5)
        
        self.thread_listbox = tk.Listbox(
            thread_frame,
            font=("Courier", 9),
            height=4,
            bg="#fafafa",
            relief=tk.SUNKEN,
            bd=2
        )
        self.thread_listbox.pack(fill=tk.X, padx=5, pady=5)
        
        # Log Section
        log_frame = tk.LabelFrame(
            main_frame,
            text="Process Log",
            font=("Arial", 12, "bold"),
            bg="#f0f0f0",
            padx=10,
            pady=10
        )
        log_frame.pack(fill=tk.BOTH, expand=True)
        
        self.log_area = scrolledtext.ScrolledText(
            log_frame,
            font=("Courier", 9),
            height=10,
            wrap=tk.WORD,
            bg="#fafafa"
        )
        self.log_area.pack(fill=tk.BOTH, expand=True)
        
        # Start thread monitoring
        self.update_thread_list()
    
    def update_thread_list(self):

        self.thread_listbox.delete(0, tk.END)
        
        # Get all active threads
        active_threads = threading.enumerate()
        
        self.thread_listbox.insert(tk.END, f"Total Active Threads: {len(active_threads)}")
        self.thread_listbox.insert(tk.END, "-" * 50)
        
        for thread in active_threads:
            thread_info = f"• {thread.name} (ID: {thread.ident}, Alive: {thread.is_alive()})"
            self.thread_listbox.insert(tk.END, thread_info)
        
        # Schedule next update
        self.root.after(100, self.update_thread_list)
    
    def log(self, message):
        self.log_area.insert(tk.END, f"{message}\n")
        self.log_area.see(tk.END)
        self.root.update_idletasks()
    
    def generate_random(self):
        size = random.randint(10, 20)
        # Ensure even size for equal division
        if size % 2 != 0:
            size += 1
        array = [random.randint(1, 100) for _ in range(size)]
        self.entry.delete(0, tk.END)
        self.entry.insert(0, ", ".join(map(str, array)))
    
    def clear_all(self):
        self.original_label.config(text="")
        self.sublist1_label.config(text="")
        self.sublist2_label.config(text="")
        self.sorted_label.config(text="")
        self.log_area.delete(1.0, tk.END)
        self.canvas.delete("all")
    
    def draw_visualization(self):
        self.canvas.delete("all")
        
        # Get canvas dimensions
        width = self.canvas.winfo_width()
        height = self.canvas.winfo_height()
        
        if width <= 1:
            width = 800
        if height <= 1:
            height = 300
        
        # Draw original list
        y_pos = 40
        self.canvas.create_text(width/2, y_pos, text="Original List", font=("Arial", 12, "bold"))
        self.canvas.create_rectangle(width/2 - 150, y_pos + 20, width/2 + 150, y_pos + 50, 
                                     fill="#ecf0f1", outline="black", width=2)
        self.canvas.create_text(width/2, y_pos + 35, 
                               text=str(self.original_list), font=("Arial", 9))
        
        # Draw arrows to sublists
        self.canvas.create_line(width/2 - 80, y_pos + 50, width/4, y_pos + 110, 
                               arrow=tk.LAST, width=2, fill="#2c3e50")
        self.canvas.create_line(width/2 + 80, y_pos + 50, 3*width/4, y_pos + 110, 
                               arrow=tk.LAST, width=2, fill="#2c3e50")
        
        # Draw sublists
        y_pos = 130
        # Sublist 1
        self.canvas.create_text(width/4, y_pos - 10, text="Sorting Thread 0", 
                               font=("Arial", 10, "bold"), fill="#16a085")
        self.canvas.create_rectangle(width/4 - 100, y_pos, width/4 + 100, y_pos + 40, 
                                     fill="#e8f8f5", outline="black", width=2)
        display_text = str(self.sorted_sublist1) if self.sorted_sublist1 else str(self.sublist1)
        self.canvas.create_text(width/4, y_pos + 20, text=display_text, font=("Arial", 9))
        
        # Sublist 2
        self.canvas.create_text(3*width/4, y_pos - 10, text="Sorting Thread 1", 
                               font=("Arial", 10, "bold"), fill="#d68910")
        self.canvas.create_rectangle(3*width/4 - 100, y_pos, 3*width/4 + 100, y_pos + 40, 
                                     fill="#fef9e7", outline="black", width=2)
        display_text = str(self.sorted_sublist2) if self.sorted_sublist2 else str(self.sublist2)
        self.canvas.create_text(3*width/4, y_pos + 20, text=display_text, font=("Arial", 9))
        
        # Draw arrows to merge
        self.canvas.create_line(width/4, y_pos + 40, width/2, y_pos + 90, 
                               arrow=tk.LAST, width=2, fill="#2c3e50")
        self.canvas.create_line(3*width/4, y_pos + 40, width/2, y_pos + 90, 
                               arrow=tk.LAST, width=2, fill="#2c3e50")
        
        # Draw merge thread label
        y_pos = 200
        self.canvas.create_text(width/2, y_pos, text="Merge Thread", 
                               font=("Arial", 10, "bold"), fill="#c0392b")
        
        # Draw final sorted list
        y_pos = 230
        self.canvas.create_rectangle(width/2 - 150, y_pos, width/2 + 150, y_pos + 40, 
                                     fill="#d5f4e6", outline="black", width=2)
        if self.final_sorted_list:
            self.canvas.create_text(width/2, y_pos + 20, 
                                   text=str(self.final_sorted_list), font=("Arial", 9))
        
        self.canvas.create_text(width/2, y_pos + 55, text="Sorted List", 
                               font=("Arial", 12, "bold"))
    
    def merge_sort(self, arr):
        """Merge sort algorithm"""
        if len(arr) <= 1:
            return arr
        
        mid = len(arr) // 2
        left = self.merge_sort(arr[:mid])
        right = self.merge_sort(arr[mid:])
        
        return self.merge_arrays(left, right)
    
    def merge_arrays(self, left, right):
        """Merge two sorted arrays"""
        result = []
        i = j = 0
        
        while i < len(left) and j < len(right):
            if left[i] <= right[j]:
                result.append(left[i])
                i += 1
            else:
                result.append(right[j])
                j += 1
        
        result.extend(left[i:])
        result.extend(right[j:])
        return result
    
    def sorting_thread(self, sublist, thread_id, start_index):
        """Sorting thread function"""
        thread_name = threading.current_thread().name
        self.log(f"[{thread_name}] Started (Thread ID: {threading.current_thread().ident})")
        self.log(f"[{thread_name}] Sublist: {sublist}")
        self.log(f"[{thread_name}] Start index: {start_index}")
        
        # Sort using merge sort
        sorted_sublist = self.merge_sort(sublist)
        
        # Store result with thread safety
        with self.lock:
            self.sorted_sublists[thread_id] = sorted_sublist
        
        self.log(f"[{thread_name}] Sorted: {sorted_sublist}")
        self.log(f"[{thread_name}] Completed\n")
        
        # Update display
        if thread_id == 0:
            self.sorted_sublist1 = sorted_sublist
            self.root.after(0, lambda: self.sublist1_label.config(text=str(sorted_sublist)))
        else:
            self.sorted_sublist2 = sorted_sublist
            self.root.after(0, lambda: self.sublist2_label.config(text=str(sorted_sublist)))
        
        self.root.after(0, self.draw_visualization)
    
    def merging_thread(self):
        """Merging thread function"""
        thread_name = threading.current_thread().name
        self.log(f"[{thread_name}] Started (Thread ID: {threading.current_thread().ident})")
        
        # Get sorted sublists
        sorted_sub1 = self.sorted_sublists[0]
        sorted_sub2 = self.sorted_sublists[1]
        
        self.log(f"[{thread_name}] Merging {sorted_sub1} and {sorted_sub2}")
        
        # Merge the two sorted sublists
        self.final_sorted_list = self.merge_arrays(sorted_sub1, sorted_sub2)
        
        self.log(f"[{thread_name}] Result: {self.final_sorted_list}")
        self.log(f"[{thread_name}] Completed\n")
        
        # Update display
        self.root.after(0, lambda: self.sorted_label.config(text=str(self.final_sorted_list)))
        self.root.after(0, self.draw_visualization)
        
        self.log("=" * 60)
        self.log("SORTING PROCESS COMPLETED SUCCESSFULLY!")
        self.log("=" * 60)
        
        self.root.after(0, lambda: messagebox.showinfo("Success", "Sorting completed successfully!"))
    
    def start_sorting(self):
        """Start the multithreaded sorting process"""
        try:
            # Parse input
            input_text = self.entry.get().strip()
            if not input_text:
                messagebox.showerror("Error", "Please enter an array of integers")
                return
            
            self.original_list = [int(x.strip()) for x in input_text.split(",")]
            
            if len(self.original_list) < 2:
                messagebox.showerror("Error", "Array must have at least 2 elements")
                return
            
            # Ensure even number of elements for equal division
            if len(self.original_list) % 2 != 0:
                messagebox.showwarning("Warning", 
                    "Array has odd number of elements. Adding a duplicate to make it even.")
                self.original_list.append(self.original_list[-1])
            
            # Clear previous data
            self.clear_all()
            self.sorted_sublists = {}
            self.sorted_sublist1 = []
            self.sorted_sublist2 = []
            self.final_sorted_list = []
            
            # Display original list
            self.original_label.config(text=str(self.original_list))
            
            self.log("=" * 60)
            self.log("MULTITHREADED SORTING APPLICATION")
            self.log("=" * 60)
            self.log(f"Original list: {self.original_list}")
            self.log(f"List size: {len(self.original_list)}\n")
            
            # Divide into two equal halves
            mid = len(self.original_list) // 2
            self.sublist1 = self.original_list[:mid]
            self.sublist2 = self.original_list[mid:]
            
            self.log(f"Dividing into two equal sublists:")
            self.log(f"Sublist 1 (indices 0-{mid-1}): {self.sublist1}")
            self.log(f"Sublist 2 (indices {mid}-{len(self.original_list)-1}): {self.sublist2}\n")
            
            self.sublist1_label.config(text=str(self.sublist1))
            self.sublist2_label.config(text=str(self.sublist2))
            
            # Initial visualization
            self.draw_visualization()
            
            # Run sorting in a separate thread to not block GUI
            def run_sorting():
                # Create and start sorting threads
                self.log("Creating sorting threads...\n")
                
                thread0 = threading.Thread(
                    target=self.sorting_thread,
                    args=(self.sublist1.copy(), 0, 0),
                    name="SortingThread0"
                )
                
                thread1 = threading.Thread(
                    target=self.sorting_thread,
                    args=(self.sublist2.copy(), 1, mid),
                    name="SortingThread1"
                )
                
                # Start threads
                thread0.start()
                thread1.start()
                
                # Wait for sorting threads to complete
                thread0.join()
                thread1.join()
                
                self.log("Both sorting threads have completed.\n")
                
                # Create and start merge thread
                self.log("Creating merge thread...\n")
                
                merge_thread = threading.Thread(
                    
                    target=self.merging_thread,
                    name="MergeThread"
                )
                
                merge_thread.start()
                merge_thread.join()
            
            # Start the sorting process in a background thread
            main_thread = threading.Thread(target=run_sorting)
            main_thread.daemon = True
            main_thread.start()
            
        except ValueError:
            messagebox.showerror("Error", "Please enter valid integers separated by commas")
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")


def main():
    """Main function"""
    root = tk.Tk()
    app = MultithreadedSortingApp(root)
    root.mainloop()


if __name__ == "__main__":
    main()